In [1]:
import numpy as np
import pandas as pd
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [2]:
mh_data=pd.read_csv('stemmed_data.csv')

In [3]:
X=mh_data.drop(columns='class',axis=1)
Y=mh_data['class']

In [4]:
#stemming
port_stem=PorterStemmer()
def stemming(text):
    text=text.lower()
    text=re.sub('\[.*?\]', '',text)
    text=re.sub("\\W"," ",text)
    text=re.sub('https?://\S+|www\.\S+','',text)
    text=re.sub('<.*?>+','',text)
    text=re.sub('[%s]'% re.escape(string.punctuation),'',text)
    text=re.sub('\n','',text)
    text=re.sub('\w*\d\w*','',text)
    text=text.split()
    text=[port_stem.stem(word) for word in text if not word in stopwords.words('english')]
    text=' '.join(text)
    return text

In [5]:
X=mh_data['text'].values
Y=mh_data['class'].values

In [6]:
vectorizer=TfidfVectorizer()
vectorizer.fit(X)
X=vectorizer.transform(X)

In [7]:
from sklearn import preprocessing
label_encoder=preprocessing.LabelEncoder()
mh_data['class']=label_encoder.fit_transform(mh_data['class'])

In [8]:
X_train,X_test,Y_train,Y_test= train_test_split(X,Y,test_size=0.2)

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [10]:
model=LogisticRegression(solver='newton-cg', max_iter=100000)
model.fit(X_train,Y_train)

LogisticRegression(max_iter=100000, solver='newton-cg')

In [11]:
from joblib import Parallel, delayed
import joblib

In [12]:
obj=np.array([model,vectorizer])

In [13]:
joblib.dump(obj,'saved_model.pkl')

['saved_model.pkl']

In [14]:
X_new=pd.Series(input())
print(type(X_new))
X_new=X_new.apply(stemming)

i am happy
<class 'pandas.core.series.Series'>


In [15]:
X_new=X_new.values
X_new=vectorizer.transform(X_new)
from scipy.sparse import csr_matrix
X_new=csr_matrix(X_new)
pred=model.predict(X_new)
print(pred)

['non-suicide']
